Copyright Matlantis Corp. as contributors to Matlantis contrib project

In [ ]:
# If the libraries are not enough to run the following script, please run the install command below
#!pip install lightgbm xgboost pycaret

In [1]:
import warnings
import os
warnings.filterwarnings("ignore")
from pycaret.regression import *
import pandas as pd
import pickle

## 4. Screening with pycaret
The first screening was conducted with pycaret.  
The dataset was divided by groupkfold with moledule id to compensate for the accuracy against unknown molecules.  

In [3]:
dataset_file = "dataset/qm9_vapor_r2scan_H.csv"

model_type = dataset_file.split("_")[-2]
seed = 12345
model_type

'r2scan'

In [4]:
df_all = pd.read_csv(dataset_file)
df_all.head()

,desc-0,desc-1,desc-2,desc-3,desc-4,desc-5,desc-6,desc-7,desc-8,desc-9,...,desc-248,desc-249,desc-250,desc-251,desc-252,desc-253,desc-254,desc-255,molecule_idx,value
0,0.725259,0.257291,0.238289,-0.157121,0.562056,0.041166,1.364311,0.292885,0.311306,0.320144,...,0.090678,-0.079473,0.432806,1.070554,0.012076,-0.087990,0.165963,-0.098839,m000001,31.5743
1,0.725260,0.257290,0.238290,-0.157120,0.562059,0.041163,1.364309,0.292885,0.311308,0.320145,...,0.090679,-0.079473,0.432805,1.070554,0.012075,-0.087990,0.165964,-0.098838,m000001,31.5743
2,0.725275,0.257281,0.238300,-0.157112,0.562067,0.041146,1.364298,0.292880,0.311308,0.320150,...,0.090683,-0.079477,0.432794,1.070552,0.012067,-0.087986,0.165965,-0.098826,m000001,31.5744
3,0.725273,0.257281,0.238300,-0.157114,0.562064,0.041147,1.364298,0.292879,0.311307,0.320149,...,0.090682,-0.079476,0.432795,1.070552,0.012067,-0.087986,0.165965,-0.098829,m000001,31.5744
4,0.702751,0.412856,0.481192,0.238448,0.789768,0.111927,1.303263,-0.103865,-0.169121,0.390848,...,-0.118581,-0.326023,0.321385,0.600912,0.007953,-0.082237,0.067479,0.129453,m000002,31.9746


In [5]:
reg = setup(df_all, target ="value", fold = 5, 
            ignore_features = ["molecule_idx"],
            fold_strategy="groupkfold", fold_groups="molecule_idx",
            )

,Description,Value
0,Session id,3218
1,Target,value
2,Target type,Regression
3,Original data shape,"(1208463, 258)"
4,Transformed data shape,"(1208463, 257)"
5,Transformed train set shape,"(845924, 257)"
6,Transformed test set shape,"(362539, 257)"
7,Ignore features,1
8,Numeric features,256
9,Preprocess,True


In [6]:
best_model = compare_models(sort="MAE", n_select=3, 
                            include=["knn", "xgboost", "lightgbm", "lr", "ridge", "lasso", "mlp"])
best_model

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.1263,0.0381,0.1953,0.9903,0.0067,0.0044,463.1520
mlp,MLP Regressor,0.1435,0.0374,0.1934,0.9905,0.0066,0.0050,293.9660
xgboost,Extreme Gradient Boosting,0.2409,0.1070,0.3270,0.9728,0.0112,0.0084,34.8340
lr,Linear Regression,0.2739,0.1364,0.3693,0.9654,0.0128,0.0096,6.3460
ridge,Ridge Regression,0.2739,0.1364,0.3693,0.9654,0.0128,0.0096,3.7460
lightgbm,Light Gradient Boosting Machine,0.2801,0.1442,0.3797,0.9634,0.0131,0.0098,73.4020
lasso,Lasso Regression,1.5081,3.9387,1.9846,-0.0000,0.0700,0.0546,3.1180


[KNeighborsRegressor(n_jobs=-1),
 MLPRegressor(max_iter=500, random_state=3218),
 XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=-1, num_parallel_tree=None, ...)]

In [7]:
os.makedirs("models", exist_ok=True)
with open(f'models/pycaret_model_compared_result_{model_type}.pkl', 'wb') as f:
    pickle.dump(best_model, f)